**Multiarmed Bandits: Optimistic Greedy versus Realistic epsilon-Greedy algorithms**

In this notebook, we use OpenAI Gym to implement optimistic and realistic initializations

In [1]:
import gym

Unfortunatley, Gym does not provide a bandit environment so we need to import it, lets install one with the command below:

In [ ]:
!git clone https://github.com/JKCooper2/gym-bandits.git > /dev/null 2>&1
!pip install /content/gym-bandits/. > /dev/null 2>&1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's import needed packages

In [ ]:
import gym_bandits
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

Define the epsilon-Greedy actions (Note: if epsilon=0 it corresponds to the Greedy action)

In [ ]:
def get_action(Q, e):
    if random.random() < e:
        return random.randint(0, 9)
    else:
        return np.argmax(Q)

We want to compare the Average Reward over repeated experiments for different values of epsilon.

We define a function that compute the average of obtained reward over repeated experiments.

In [ ]:
def average(tot_rewards):
    avg = []
    for i in range(len(tot_rewards[0])):
        temp = 0
        for j in range(len(tot_rewards)):
            temp += tot_rewards[j][i]
        temp /= len(tot_rewards)
        avg.append(temp)
    return avg

Main code: define an Environment for 10-armed Bandit whose rewards are normal distributed with standard deviation = 1 and a mean drawn from a normal distribution (0,1). Same as Sutton&Barto book

In [ ]:
k = 10  # Number of actions (arms)
iter = 1000 # Number of steps
repeat = 2000 # Number of repetitions
o0 = []  # Results for Optimistic Greedy algorithm
e01 = [] # Results for epsilon = 0.1

np.random.seed(42)
env = gym.make("BanditTenArmedGaussian-v0") # define Environment

Run the Optimistic Greedy algorithm (epsilon = 0) several times to average the results

In [ ]:
env.reset()
tot_rewards = []
e = 0
for _ in tqdm(range(repeat)):
    Q = 5 * np.ones(shape=(k))
    N = np.zeros(shape=(k))
    rewards = []
    for i in range(iter):
        action = get_action(Q, e)
        state, reward, done, info = env.step(action)
        rewards.append(reward)
        N[action] += 1
        Q[action] = Q[action] +  1/N[action] * (reward - Q[action])
    env.reset()
    tot_rewards.append(rewards)

o0 = average(tot_rewards)

Run the realistic epsilon-Greedy algorithm with epsilon = 0.01 several times to average the results

In [ ]:
env.reset()
tot_rewards = []
e = 0.1
for _ in tqdm(range(repeat)):
    Q = np.zeros(shape=(k))
    N = np.zeros(shape=(k))
    rewards = []
    for i in range(iter):
        action = get_action(Q, e)
        state, reward, done, info = env.step(action)
        rewards.append(reward)
        N[action] += 1
        Q[action] = Q[action] +  1/N[action] * (reward - Q[action])
    env.reset()
    tot_rewards.append(rewards)

e01 = average(tot_rewards)

Finally, we plot the results to compare the Greedy algorithm against the epsilon-Greedy algorithm

In [ ]:
plt.plot(o0, color='blue', label='Optimistic Init, e = 0')
plt.plot(e01, color='olive', label='0 init, e = 0.1')
plt.xlabel("Steps")
plt.ylim(0, max(o0) + 0.2)
plt.ylabel("Average Reward")
plt.title("Average Reward vs. Steps on 10 Armed Bandit")
plt.legend()
plt.show()
